In [3]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa

from shutil import copyfile

# from matplotlib import pyplot as plt
from matplotlib.image import imread, imsave
# %matplotlib inline 

import os

dir_ = '../data/v01/ds'
output_dir = '../data/v02/ds'

filters = [
#     iaa.MotionBlur(k=11, angle=180, name='mba180k11'),
#     iaa.MotionBlur(k=11, angle=90, name='mba90k11'),
#     iaa.GaussianBlur(sigma=(0.0, 4.0), name='gbs1'),
#     iaa.GammaContrast((0.5, 0.5), name='gc0.5'),
    iaa.GammaContrast((2.0, 2.0), name='gc2.0')
]

def load_image(infilename) :
    try:
        return imread(infilename)
    except IOError:
        print(f'Could not read {infilename}')
        return None

def save_image(npdata, outfilename):
    imsave(outfilename, npdata)

    
images = dict()

for root, dirs, files in os.walk(dir_):
    for file in files:
        if not file.endswith(".json"):
            images[file] = load_image(os.path.join(root, file))

augmented = [filter_(images=images.values()) for filter_ in filters]

for idx, file in enumerate(images.keys()):
    copyfile(os.path.join(dir_, file), os.path.join(output_dir, file))
    copyfile(os.path.join(dir_, file + '.json'), os.path.join(output_dir, file + '.json'))
    for filter_idx in range(len(filters)):
        aug_file = f'{filters[filter_idx].name}-{file}'
        try:
            save_image(augmented[filter_idx][idx], os.path.join(output_dir, aug_file))
            copyfile(os.path.join(dir_, file + '.json'), os.path.join(output_dir, aug_file + '.json'))
        except ValueError:
            print(f'Could not save {aug_file}')

# plt.imshow(list(images.values())[0])
# plt.imshow(augmented[4][0])
# plt.imshow(augmented[1][0])
# plt.show()